Turn chicken scratches into module.

Reference:
-http://www.cnccookbook.com/CCCNCGCodeG20G21MetricImperialUnitConversion.htm
- http://marlinfw.org/meta/gcode/
- https://github.com/grbl/grbl/wiki
- http://www.linuxcnc.org/docs/2.5/html/gcode/other-code.html

#### V0.06

In [ ]:
class GCode(object):
    def __init__(self):
        self.linenumbering = False
        self.buffer = list()
        
    @property
    def code(self):
        if self.linenumbering:
            pass
        return "\n".join(self.buffer)
        
    def save(self, filename):
        pass
    
    def send(self):
        pass
    
    def __str__(self):
        return self.code

    def __repr__(self):
        return "<GCode>[cmds={}]".format(len(self.buffer))
    
    def _repr_html_(self):
        html = list()
        for cmd_line in self.buffer:
            cmd, *args = cmd_line.split(" ")
            html_line = "<b>{cmd}</b> <i>{args}</i>".format(cmd=cmd, args=" ".join(args))
            html.append(html_line)
        return "<br>\n".join(html)
    

def cmd_factory(cmd, doc=None):
    def cmd_fcn(self, **kwargs):
        args = list()
        for key, value in kwargs.items():
            args.append("{key}{value}".format(key=key, value=value))
        
        cmd_str = "{cmd} {args}".format(cmd=cmd, args=" ".join(args))
        self.buffer.append(cmd_str)  
    return cmd_fcn      

# Good core to start with.
commands=list()
# GCodes
for code in [0, 1, 2, 3, 4, 20, 21, 28, 90, 91]:
    commands.append("G{code}".format(code=code))
# MCodes
for code in [0, 1, 2, 3, 4, 5, 6]:
    commands.append("M{code}".format(code=code))
# Commands to implement.
print(commands)

for command in commands:
    setattr(GCode, command, cmd_factory(command))

In [ ]:
from GRBL import GRBL
cnc = GRBL("/dev/cnc_3018")

In [ ]:
cnc = GRBL("/dev/cnc_3018")

In [ ]:
cnc.reset()

In [ ]:
cnc.laser_mode

In [ ]:
def square(size=20):
    program = GCode()
    program.G21()
    program.G91()
    program.G1(F=200)
    program.M4(S=25)
    program.G1(X=size)
    program.G1(Y=size)
    program.G1(X=-size)
    program.G1(X=-size)
    program.M5()
    return program

In [ ]:
square(20)

In [ ]:
sqr = square(20)

In [ ]:
print(sqr)

In [ ]:
from time import sleep

In [ ]:
for line in str(sqr).splitlines():
    cnc.cmd(line)
    sleep(0.2)

Figuring out buffering / streaming.

In [1]:
from GRBL import GRBL
cnc = GRBL("/dev/cnc_3018")

In [3]:
buffer = list()
buffer.append(cnc.write("G21"))
buffer.append(cnc.write("G91"))
buffer.append(cnc.write("G0 F100"))

In [6]:
sum([sum(buf) for buf in buffer])

19

In [8]:
data = cnc.read(multiline=True)
data

['ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok']

In [9]:
len(data)

10

In [10]:
len(buffer)

3

In [23]:
def stream(self, program, compact=True):
    if isinstance(program, str):
        program = program.splitlines()
    elif isinstance(program, list):
        pass    
    else:
        raise(Exception("Unsupported: {}".format(type(program))))
                          
    return "Foo"
setattr(GRBL, "stream", stream)

cnc = GRBL("/dev/cnc_3018")

In [24]:
cnc.stream('')

'Foo'

In [25]:
cnc.stream([])

'Foo'

In [26]:
cnc.stream(dict())

Exception: Unsupported: <class 'dict'>

In [35]:
from time import sleep

In [36]:
def stream(self, program, compact=True):
    if isinstance(program, str):
        program = program.splitlines()
    elif isinstance(program, list):
        pass    
    else:
        raise(Exception("Unsupported: {}".format(type(program))))
    
    # Strip whitespace and force letters to capital.
    program = [line.strip().upper() for line in program]
    # Save bits.
    if compact:
        program = [line.replace(" ", "") for line in program]
        
    self.serial.flushInput()
    for program_line in program:
        cnc.write(program_line)
        result = cnc.read(multiline=True, timeout=0.1)
        while len(result)==0:
            print("Buffer Full, waiting...")
            sleep(0.5)
            result = cnc.read(multiline=True, timeout=0.1)
setattr(GRBL, "stream", stream)

cnc = GRBL("/dev/cnc_3018")

In [39]:
program = list()
program.append("G21")
program.append("G91")
for i in range(200):
    program.append("G0 X1")

In [41]:
cnc.stream(program)

In [ ]:
cnc.write("G0 X1")

In [ ]:
cnc.read(timeout=0.5)

In [ ]:
cnc.write("G0 X10")